In [1]:
#Dependencies
import os
import io
import numpy as np
import glob
import PIL


import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

import keras 
from keras.preprocessing import image
from keras.models import Model
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import (inception_v3, preprocess_input, decode_predictions)

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Load model, exclude top layer 
baseModel = inception_v3.InceptionV3(include_top=False, 
                         weights='imagenet')

In [3]:
# Add the last layer 
num_classes = 8
x = baseModel.output
x = GlobalAveragePooling2D()(x)
# add a Fully Connected Dense Layer of size 1024
x = Dense(1024, activation='relu')(x)
# add softmax on the output 
predictions = Dense(num_classes, activation='softmax')(x)
# new keras model with last layer 
model = Model(input=baseModel.input, output=predictions)

C:\Users\rupal\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [4]:
# Freeze all layers from basemodel to exclude from training
for layer in baseModel.layers:
    layer.trainable = False
# compile the new model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',  
              metrics=['accuracy'])  

In [4]:
train_dir = '/floydhub/input/images/train'
valid_dir = '/floydhub/input/images/valid' 

In [5]:
target_size = (299, 299)
train_batch_size = 100
valid_batch_size = 8

In [6]:
# Perform data augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      horizontal_flip=True)

In [7]:
#grabs images from the folders

train_batches = train_datagen.flow_from_directory(
                                directory=train_dir,
                                target_size=target_size, 
                                classes=['animals','baches','birds','electronics','fashion','nature','landmarks','people'], 
                                batch_size=train_batch_size 
                                )

valid_datagen = ImageDataGenerator(rescale=1. / 255)
valid_batches = valid_datagen.flow_from_directory(
                                directory=valid_dir,
                                target_size=target_size,  
                                classes=['animals','beaches','birds','electronics','fashion','nature','landmarks','people'],                             
                                batch_size=valid_batch_size 
                                )    

Found 1409 images belonging to 8 classes.
Found 32 images belonging to 8 classes.


In [8]:
epochs = 10

steps_per_epoch = len(train_batches.filenames)/train_batch_size

In [9]:
#fit the model on data generator batch by batch
history = model.fit_generator(train_batches, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_data=valid_batches, 
                    validation_steps=8, 
                    epochs=epochs, 
                    verbose=2)

In [10]:
# Save trained model
model.save('inception3_train_ep6.h5')

In [ ]:
# Check performance by looking at the loss and accuracy curves.

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()